###  앙상블 학습

* 앙상블 학습의 유형은 보팅, 배깅, 부스팅 세가지로 나눌 수 있으며 이외에도 스태깅을 포함한 다양한 앙상블 방법이 있다.
* 보팅의 경우 서로 다른 알고리즘을 가진 분류기를 결합하는 것이고 배깅의 경우 각각의 분류기가 모두 같은 유형의 알고리즘 기반이다.
* 정형 데이터의 예측 분석 영역에서는 매우 높은 예측 성능. Bagging 과 Boosting
    - bagging 방식은 같은 알고리즘을 사용해서 보팅 후 최종 예측값을 예측 (의사결정나무를 여러개를 뽑아서 예측을 수행 -> 그 결과를 가지고 보팅을 해서 최종 예측값을 정함)</br></br>
* 배깅 방식의 대표인 Random Forest는 뛰어난 예측 성능, 상대적으로 빠른 수행시간, 유연성 등으로 애용.
* 부스팅의 효시는 Gradient Boosting, 한 단계 발전시키면서도 시간 단축시킨 XgBoost, LightGBM이 정형 데이터의 분류 영역에서 
  활용도 확대
* 앙상블의 앙상블이라고 불리는 스태킹 기법
* 앙상블의 기본 알고리즘은 결정 트리

##### Voting Classifier
- 하드 보팅 : 다수결 원칙, 다수의 분류기가 결정한 예측값을 최종 보팅 결과값으로 선정
- 소프트 보팅 : 분류기들의 레이블 값 결정 확률을 모두 더해서 평균하고 이들 중 가장 높은 레이블 값을 최종 보팅 결과값으로 선정

In [3]:
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

cancer = load_breast_cancer()
data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


In [4]:
# 보팅 적용을 위한 개별 모델은 로지스틱 회귀와 KNN 객체 생성
# Estimators에 넣어줌
logistic_regression = LogisticRegression() 
knn = KNeighborsClassifier(n_neighbors=9)


# 개별모델을 소프트보팅 기반의 앙상블 모델로 구현한 분류기
# 보팅분류기 생성
vo_clf = VotingClassifier(estimators=[('LR' ,logistic_regression),('KNN',knn)],voting='soft')

# 데이터를 훈련 데이터와 테스트 데이터로 나누기
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2, random_state=156)

# 보팅 분류기의 학습/예측/평가
vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print(f'Voting 분류기 정확도 :{accuracy:.3f}')

# 개별 모델의 학습/예측/평가
classifiers = [logistic_regression, knn]
for classifier in classifiers:
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    class_name = classifier.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test, pred)))

# 각각의 개별 알고리즘보다 Voting 분류기의 일반적으로 정확도가 높다 --> 항상 그런건 아님

Voting 분류기 정확도 :0.947
LogisticRegression 정확도: 0.9386
KNeighborsClassifier 정확도: 0.9649


### RandomForest
- 참조 url https://eunsukimme.github.io/ml/2019/11/26/Random-Forest/

### RandomForest 
- 수많은 의사 결정 트리(Decision Tree)가 모여서 생성됨
- Classification(분류) 및 Regression(회귀) 문제에 모두 사용 가능
- Missing value(결측치)를 다루기 쉬움
- 대용량 데이터 처리에 효과적
- 모델의 노이즈를 심화시키는 Overfitting(오버피팅) 문제를 회피하여, 모델 정확도를 향상시킴
- Classification 모델에서 상대적으로 중요한 변수를 선정 및 Ranking 가능
- 편향은 유지시키고 분산은 감소하므로써 예측력 증가 과적합 위험 감소</br></br>

### 주요 하피어 파라미터
- n_estimators: 랜덤 포레스트 안의 결정 트리 갯수
    - n_estimators는 클수록 좋습니다.
    - 결정 트리가 많을수록 더 깔끔한 Decision Boundary가 나오겠죠.
    - 하지만 그만큼 메모리와 훈련 시간이 증가합니다.
    - Default는 10입니다.</br></br>
- max_features: 무작위로 선택할 Feature의 개수
    - max_features=n_features이면 30개의 feature 중 30개의 feature 모두를 선택해 결정 트리를 만듭니다.
    - 단, bootstrap=True이면 30개의 feature에서 복원 추출로 30개를 뽑습니다.
    - 특성 선택의 무작위성이 없어질 뿐 샘플링의 무작위성은 그대로인 것입니다.
    - bootstrap=True는 default 값입니다.
    - 따라서 max_features 값이 크다면 랜덤 포레스트의 트리들이 매우 비슷해지고, 가장 두드러진 특성에 맞게 예측을 할 것입니다.
    - max_features 값이 작다면 랜덤 포레스트의 트리들이 서로 매우 달라질 것입니다. 따라서 오버피팅이 줄어들 것입니다.
    - max_features는 일반적으로 Defalut 값을 씁니다.

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

# features.txt 파일에는 피처 이름 index와 피처명이 공백으로 분리되어있음, 이를 DataFrame으로 로드
feature_name_df = pd.read_csv('dataset/human_activity/features.txt', sep='\s+', header=None, names =['column_index', 'column_name'])    

print(feature_name_df.head())

# 피처명 index를 제거, 피처명만 리스트 객체로 생성
feature_name = feature_name_df.iloc[:,1].values.tolist() # 행쪽은 전부 열은 column 이름만 추출
print(feature_name[:10])

   column_index        column_name
0             1  tBodyAcc-mean()-X
1             2  tBodyAcc-mean()-Y
2             3  tBodyAcc-mean()-Z
3             4   tBodyAcc-std()-X
4             5   tBodyAcc-std()-Y
['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z', 'tBodyAcc-max()-X']


In [6]:
feature_dup_df = feature_name_df.groupby('column_name').count()
print(feature_dup_df[feature_dup_df['column_index']>1].count())
feature_dup_df[feature_dup_df['column_index']>1].head()

column_index    42
dtype: int64


,column_index
column_name,
"fBodyAcc-bandsEnergy()-1,16",3
"fBodyAcc-bandsEnergy()-1,24",3
"fBodyAcc-bandsEnergy()-1,8",3
"fBodyAcc-bandsEnergy()-17,24",3
"fBodyAcc-bandsEnergy()-17,32",3


In [7]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                 columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index() 
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(),feature_dup_df,how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name',
                                                             'dup_cnt']].apply(lambda x:x[0]+'_'+str(x[1])
                                                                              if x[1]>0 else x[0], axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'],axis=1)
    return new_feature_name_df 

In [8]:
# dataframe만드는 사용자함수
def get_human_dataset():
    feature_name_df = pd.read_csv('dataset/human_activity/features.txt', sep='\s+', header=None, \
                              names = ['column_index','column_name'])
    
    feature_name = feature_name_df.iloc[:,1].values.tolist()
    
    # X_train과 X_test 데이터세트를 만들어줌
    X_train  = pd.read_csv('dataset/human_activity/train/X_train.txt', sep='\s+', header=None)
    X_train.columns = feature_name
    X_test  = pd.read_csv('dataset/human_activity/test/X_test.txt', sep='\s+', header=None)
    X_train.columns = feature_name
    
    y_train = pd.read_csv('dataset/human_activity/train/y_train.txt', sep='\s+', header=None, names=['action'])
    y_test = pd.read_csv('dataset/human_activity/test/y_test.txt', sep='\s+', header=None, names=['action'])
    
    return X_train, X_test, y_train, y_test  # input 없고 output은 존재하는 사용자 함수
X_train, X_test, y_train, y_test = get_human_dataset()

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

# 결정 트리에서 사용한 get_human_dataset()를 이용해 학습/테스트용 DataFrame 반환
X_train, X_test, y_train, y_test = get_human_dataset()

# 랜덤 포레스트 학습 및 별도의 테스트 세트로 예측 성능 평가
rf_clf = RandomForestClassifier(random_state=0) # 객체 생성
rf_clf.fit(X_train, y_train) # 학습
pred = rf_clf.predict(X_test) # 예측
accuracy = accuracy_score(y_test, pred)
print(f'rf 정확도 : {accuracy :.3f}')

rf 정확도 : 0.925


In [10]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[50, 100], # 값을 늘릴수록 학습 수행 시간이 오래 걸림
    'max_depth' : [],
    'min_samples_leaf':[],
    'min_samples_split':[]
}

# rf 로 객체 생성 후 GridSearchCV 수행

In [13]:
from sklearn.model_selection import GridSearchCV

params= {
    'n_estimators':[50,100],
    'max_depth':[6,8,10,12],
    'min_samples_leaf':[8,12,18],
    'min_samples_split':[8,16,20]
}

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state=0,n_jobs=-1)
grid_cv = GridSearchCV(rf_clf,param_grid =params,cv=2,n_jobs=-1)
grid_cv.fit(X_train,y_train) # 훈련데이터로 한것 cv=2로 교차 검증까지함  
# 이것을 이제 최적 하이퍼파라미터 적용해서 -> 테스트 데이터에 적용시켜야함 

print(f'최적하이퍼파라미터: \n {grid_cv.best_params_}')
print(f'최고 예측 정확도: \n {grid_cv.best_score_}')

# n_jobs=-1로 지정해주면 모든 코어를 다 사용하기때문에 컴퓨터는 뜨거워지겠지만, 속도는 많이 빨라집니다.

#GridSearchCV의 iteration시마다 수행 결과 메시지를 출력합니다.
#verbose=0(default)면 메시지 출력 안함
#verbose=1이면 간단한 메시지 출력
#verbose=2이면 하이퍼 파라미터별 메시지 출력

최적하이퍼파라미터: 
 {'max_depth': 10, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도: 
 0.9179815016322089


In [ ]:
# 최적 하이퍼파라미터 적용해서 -> 테스트 데이터에 적용 
y_pred = grid_cv.best_estimator_.predict(X_test)
accuracy_test = accuracy_score(y_test , y_pred)
print('테스트 데이터에서 정확도: {0:.3f}'.format(accuracy_test))

In [ ]:
# 변수 중요도 시각화
import seaborn as sns
import matplotlib.pyplot as plt

best_dclf = grid_cv.best_estimator_

# feature_importances_ 먼저 뽑아줌
ftr_importances_values = best_dclf.feature_importances_

# 시리즈 변환
ftr_importances = pd.Series(ftr_importances_values, index=X_train.columns)

# 20개 추출
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]

plt.figure(figsize = (8,6))
plt.title('top20')
sns.barplot(x=ftr_top20, y=ftr_top20.index)
plt.show()

### GBM(Gradient Boosting Machine)

- 부스팅 알고리즘은 여러 개의 약한 학습기(weak learner)를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가면서 학습하는 방식
- 가중치 업데이트를 경사 하강법(Gradient Descent)를 이용한다.
    - 경사하강법 원리 : 손실(비용)을 최소화할 수 있도록 가중치의 업데이트 값을 도출
- 분류는 물론이고 회귀도 가능(랜덤포레스트와 똑같음 GBM계열도 회귀 사용가능)
- 파라미터 : n_estimators, max_depth, max_features(속성값(컬럼)을 최대로 제한해줌 
    - loss : 경사하강법에서 사용할 비용함수 지정. 기본값 deviance 적용
    - learning_rate : GBM이 학습할 때마다 적용할 학습률.
            - 오류값 보정 시 적용하는 계수로 0 ~ 1 사이의 값 지정 -> 기본값은 0.1
            - 너무 작은 값을 설정하면 예측 성능이 높아지나 수행시간이 오래 걸리고 너무 큰 값을 적용하면 최소 오류 값을 찾지 못하고 그냥 지나처 버려 예측 성능이 떨어질 가능성이 높아지지만 빠른 수행이 가능하다
            - 이러한 특성 때문에 learning_rate는 n_estimator와 상호 보완적으로 조합해 사용해야 한다
            - learning_rate는 작게 하고 n_estimaators를 크게 하면 더 이상 성능이 좋아지지 않는 한계점까지 예측 성능이 조금씩 좋아질 수 있다. 하지만 수행 시간이 너무 오래걸리는 단점이 있고 성능도 현격히 좋아지지는 않는다
    - n_estimator : weak learner의 개수 기본값 100
    - subsample : weak learner가 학습에 사용하는 데이터의 샘플링 비율. 기본값은 1

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time

X_train, X_test, y_train, y_test = get_human_dataset()

# GBM 수행 시간 측정을 위함-> 시작 시간 설정.
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test,gb_pred)

print(f'GBM 정확도: {gb_accuracy:.3f}')
print(f'GBM 수행 시간: {time.time() - start_time}')

GBM 정확도: 0.939
GBM 수행 시간: 584.7644472122192


In [ ]:
# [과제] 상기 케이스에 대하여 성능개선을 위한 튜닝을 수행하세요.
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100,500],
    'learning_rate':[0.05,0.1]
}

# GridSearchCV의 iteration시마다 수행 결과 메시지를 출력합니다.
# verbose=0(default)면 메시지 출력 안함
# verbose=1이면 간단한 메시지 출력
grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1)
grid_cv.fit(X_train, y_train)

print('최적 하이퍼 파라미터 :\n', grid_cv.best_parmas_)
print('최고 예측 정확도 : {0:.4f}'.format(grid_cv.best_score_))

Fitting 2 folds for each of 4 candidates, totalling 8 fits


In [ ]:
# GridSearchCV를 이용해 최적으로 학습된 estimator로 예측 수행
# GBM은 과적합에도 뛰어난 예측 성능을 가진 알고리즘-> 하지만 수행시간이 오래 걸리는 단점
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도 : {0:.4f}'.format(gb_accuracy))